#Model Hub Login config (ToDO)

Import /install libraries

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers[torch]
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 27.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data sets
Load data from CSV files for test and dev as well as surpisal sorted csv file for train

custom dataset class? https://www.youtube.com/watch?v=PXOzkkB5eH0

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict
def create_ds_dict(ds_dir):
  '''args:str ds_dir: fpath to directory with sentence CSV files
          str train_type: rand or curriculum
          lst int seeds: list of seeds for shuffling dataset splits
     return DatasetDict ds_dict where {k-->ds_split:v-->ds with sentIDs and sentences}'''
  #datFiles={"train":(ds_dir+"babylm_sent_train.csv"),"validation":(ds_dir+"babylm_sent_dev.csv"),"test":(ds_dir+"babylm_sent_test.csv")}
  csv_dict=load_dataset("csv", data_files="/content/curriculum_lstmrev_sumsurp_root2.0_T150000.csv", sep="\037") #csv_dict=load_dataset("csv", data_files=datFiles, sep="\037") #load train test dev split into a Datasetdictionary
  ds_dict=DatasetDict()
  for k,v in csv_dict.column_names.items(): #remove unecessary columns
    ds_dict=csv_dict.remove_columns([col for col in v if col!='sentid'and col!='sentence'])
  #shuffle data , according to train_type
  '''if train_type=="rand":
    print("pre-shuffle train:",ds_dict["train"]["sentence"][1])
    ds_dict["train"].shuffle(seed)
    print("post-shuffle train",ds_dict["train"]["sentence"][1])
  print("pre-shuffle valid:",ds_dict["validation"]["sentence"][1])
  ds_dict["validation"].shuffle(seed)
  print("post-shuffle valid:",ds_dict["validation"]["sentence"][1])
  print("pre-shuffle test:",ds_dict["test"]["sentence"][1])
  ds_dict["test"].shuffle(seed)
  print("post-shuffle test:",ds_dict["test"]["sentence"][1])'''
  return ds_dict


In [ ]:
curr_dict=create_ds_dict('')
print(curr_dict)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a0e29a240ab092a4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentid', 'sentence'],
        num_rows: 4800032
    })
})


#Tokenizer

###Loading pre-trained BPE

### Tokenizer components, hugging face links for component options and , GPT 2 BPE specification
1. [Normalization](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.normalizers) ( any cleanup of the text that is deemed necessary, such as removing spaces or accents, Unicode normalization, etc.):
  *   GPT2 BPE does not use a normalizer
2. [Pre-tokenization](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.pre_tokenizers) (splitting the input into words):
  *   GPT2 BPE is not dependent on significant  pre-tokenization
3. Running the input through the [model](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.models) (using the pre-tokenized words to produce a sequence of tokens) and train tokenizer using model specific [trainer](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.trainers):
  *   GPT2 BPE uses the BPE model
4. [Post-processing](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.processors) (adding the special tokens of the tokenizer, generating the attention mask and token type IDs):
  *   GPT2 BPE uses????
5. [Decoding](https://huggingface.co/docs/tokenizers/python/latest/components.html#decoders) (technically not a compondent of tokenization , but the process through which one decodes the output of tokenization):
  *   Logically, given a byte level encoder , should use a byte level decoder


In [ ]:
#tokenizer related libraries
import tokenizers
from tokenizers import (
    normalizers, #1
    Tokenizer,
    pre_tokenizers, #2
    models, #3
    trainers,#3
    processors, #4
    decoders, #5
)
from transformers import PreTrainedTokenizerFast
def create_BPE_tok(sents,save_dir):
  '''args: str list sents
           str save_dir
     returns: PreTrainedTokenizerFast wrapped tokenizer'''
  tokenizer = Tokenizer(models.BPE()) #tokenizer objects are intialized with a model argument which specifies with algorithm they will use for tokenization
  #GPT-2 does not use a normalizer, so we skip #1 and go directly to #2: pre-tokenization
  tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False) #prevents default addition of space char at the beginning of a sentence
  #3 passing input to model and training tokenizer
  trainer = trainers.BpeTrainer(vocab_size=50272, special_tokens=["<|endoftext|>", '<image>', '</c>', '<PERSON>']) # EOS is apparently the only special token for BPE (rest based on baseline github) , set vocab size according to babyLM baseline model config val
  tokenizer.train_from_iterator(sents, trainer=trainer)#tokenizer.train(["train.txt"], trainer=trainer) #can use train from iterator to file by file
  tokenizer.post_processor = processors.ByteLevel(trim_offsets=False) #4
  tokenizer.decoder = decoders.ByteLevel()#5

  ###Save (and Push?) Tokenizer
  wrapped_tokenizer = PreTrainedTokenizerFast(
      tokenizer_object=tokenizer,
      bos_token="<|endoftext|>",
      eos_token="<|endoftext|>",
  )
  #usable by HF to recreate this tokenizer
  wrapped_tokenizer.save_pretrained(save_dir+"babyLM10M-byte-level-BPE-tokenizer")
  #tokenizer.push_to_hub("babyLM10M-byte-level-BPE-tokenizer") #usable if logged into HF hub
  return wrapped_tokenizer

##Load Tokenizer

In [ ]:
from transformers import AutoTokenizer
# code for loading tokenizer from pytorch object
#tokenizer=torch.load("path to tokenizer") #likely wont work
#Code for loading tokenizer from HF wrapper (more likely to work)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/OPT 125M checkpoints/babyLM10M-byte-level-BPE-tokenizer") #Could specify path to a checkpoint or a tokenizer directory (requires that tokenizer was saved using HF)

# Get Batches (data loading)
1.   Order sentences:Shuffled during dataset laoding (depending on whether currOPT or randOPT)
2.   Tokenize and truncate sentences according to context length
3.   use data collator to take tokenized list and dynamically pad (pad to longest) as per batch size (32 sequences)


In [ ]:
import torch
from transformers import DataCollatorForLanguageModeling
import csv
def dynamically_pad_pretokenized(tokenized_batch,tokenizer):
  '''arg: batch of inputIds and corresponding tokenizer
     return: dynamically padded batch of pt tensors for causal LM'''
  tokenizer.pad_token = tokenizer.eos_token
  data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False , return_tensors="pt")
  tensor=data_collator(tokenized_batch)
  return tensor

In [ ]:
def collate_and_log_batch(tokenized_batch,tokenizer,batches_so_far,log_file):
  '''arg: batch of inputIds and corresponding tokenizer , log_file
     log: info about tensor dimensions into a csv
     return: dynamically padded batch of pt tensors for causal LM'''
  #print(len(tokenized_batch))
  collated_batch=dynamically_pad_pretokenized(tokenized_batch,tokenizer)
  #print(len(collated_batch))
  fields = collated_batch.keys()

  ###Logging#####
  if batches_so_far==0:# initialize tensor dim log csv with 1st tensor
      with open(log_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Key', 'Size 1', 'Size 2'])
        for k, v in collated_batch.items():
          writer.writerow([k, v.size(0), v.size(1)])
  else:
      with open(log_file, 'a', newline='') as file:  # Open the file in append mode
            writer = csv.writer(file)
            for k, v in collated_batch.items():
              writer.writerow([k, v.size(0), v.size(1)])
  return collated_batch

In [ ]:
from tqdm.notebook import tqdm
def create_tokenized_batches(sents,tokenizer,context_length,batch_size,log_file):
  '''Args: str list sents: ordered list of sentences to iterate over
           tokenizer
           context_length: num of previous tokens that can be used to predict next token in pretraining
           batch_size: num sequences of len<=context_length that can be processed by model in a step
           log_file: CSV file to log tensor dimensions to
      return:array of pt tensors containing dynamically padded batches'''
  collated_batches=[]
  batch=[]
  for count,sent in tqdm(enumerate(sents, start=1),total=len(sents)):
    outputs = tokenizer(
        sent,
        truncation=True, #chunk sentence according to context length if too long
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    for input_id in outputs["input_ids"]:#input chunks count as seperate sequences
      if len(batch)<batch_size:
        batch.append(input_id)
      else: #batch_size reached ,  collate into tensor
        collated_batch=collate_and_log_batch(batch,tokenizer,len(collated_batches),log_file)
        collated_batches.append(collated_batch)
        #print(len(batch))
        batch=[]
        batch.append(input_id)
  #add last batch (potentially smaller than batch size)
  collated_batch=collate_and_log_batch(batch,tokenizer,len(collated_batches),log_file)
  collated_batches.append(collated_batch)
  print("batches created:",len(collated_batches))
  return collated_batches

In [ ]:
curr_sents=curr_dict["train"]["sentence"]
curr_dl=create_tokenized_batches(curr_sents,tokenizer,128,32,"root2_batch_log.csv")

  0%|          | 0/4800032 [00:00<?, ?it/s]

batches created: 150008


In [ ]:
print(tokenizer.tokenize(curr_sents[0]))
print(tokenizer.tokenize(curr_sents[1]))

['-', 'ĠWhat', '?']
['I', 'Ġdon', "'t", 'Ġknow', '.']


#OPT 125M model

##Configuration and initialization
Intialize an OPT model to closely replicat OPT125M architecture , using details from [(Zhang et al., 2022) ](https://https://arxiv.org/pdf/2205.01068.pdf) and [babyLM OPT125M vaseline](https://huggingface.co/babylm/opt-125m-strict-small/blob/main/config.json). See further details on the 15 params at huggingface OPTconfig documentation


In [ ]:
from transformers.models.opt import OPTConfig
from transformers.models.opt.modeling_opt import OPTForCausalLM
def init_OPT_CLM():
  '''initialize a OPT 125M model with a 0.2M param causal language modelling head for pre-training , according configuration params rationalized in comments'''
  #Set config params
  config=OPTConfig()
  print(config) #check defaults
  config.vocab_size=50272 #vocab size used in babylm baseline
  config.num_hidden_layers=768 #See dLayers , table 1 zhang et Al.
  config.num_hidden_layers=12 #See #L , table 1 zhang et Al.
  config.ffn_dim=3072 #see OPT125m baseline
  config.num_attention_heads=12 #See #H , table 1 zhang et Al.
  config.activation_function="relu"#see zhang et al
  config.max_position_embeddings=2048 #see OPT125m baseline
  config.do_layer_norm_before=True #see OPT125m baseline
  config.word_embed_proj_dim=768 #generally equal to hidden size , intialized accoring to OPT125M baseline
  config.dropout=0.1 # see Zhang et Al : "We use a dropout of 0.1 throughout, but we do not apply any dropout to embeddings."
  config.attention_dropout=0.0 #see OPT125M baseline
  config.init_std=0.02 #Intialized according to OPT125M baseline however potnetially signifcant differences from zhang et Al: “For weight initialization ….. using a normal distribution with zero mean and standard deviation of 0.006.”” Standard deviation for output layers are scaled by a 1.0/ √ 2L term where L is the total number of layers. ”
  config.use_cache=True #see OPT125M baseline
  config.enable_bias=True #see OPT125M baseline
  config.layer_norm_elementwise_affine=True #see OPT125M baseline
  print(config) #check updates
  #intialize OPTm model with a causalLM head for training
  OPTCLM=OPTForCausalLM(config)
  #check architecture
  print(OPTCLM)
  model_size = sum(t.numel() for t in OPTCLM.parameters())
  print(f"OPT size: {model_size/1000**2:.1f}M parameters")
  return OPTCLM

## Optimizer and Scheduler

OPT paper:"We follow a linear learning
rate schedule, warming up from 0 to the maximum
learning rate over the first 2000 steps in OPT-175B,
or over 375M tokens in our smaller baselines, and
decaying down to 10% of the maximum LR over
300B tokens. A number of mid-flight changes
to LR were also required (see Section 2.5). Our
batch sizes range from 0.5M to 4M depending on
the model size (see Table 1) and is kept constant
throughout the course of training."

Platanios et al:
*   for large experiments with tranformers T=50K (see curriculum hyperparameters) (large dataset was WMT with 4.5M sentences)
*   Twarmup  "is set to 10,000 in these experiments" => 1/5 of steps
*    "This schedule was proposed in the original
Transformer paper (Vaswani et al., 2017)"

Vaswani et al., 2017:
* "We trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the
bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps
(3.5 days)."
* Also used linear learning rate "We used warmup_steps = 4000." => small fraction
*Also had the same large WMT 4.5M sentence dataset


In [ ]:
from transformers import get_scheduler
from transformers import AdamW
def create_optimizer_and_lr_scheduler(total_training_steps):
  '''args: int total_training_steps: int epochs*int batches per epoch
     returns AdamW optimizer , linear learning rate scheduler (configured as per  OPT paper) '''
  optimizer = AdamW(model.parameters(), lr=5e-5,betas=(0.9,0.95),weight_decay=0.1) #Zhang et Al "We use an AdamW optimizer (Loshchilov andHutter, 2017) with (β1, β2) set to (0.9, 0.95), and weight decay of 0.1"
  lr_scheduler = get_scheduler(
      name="linear", #zhang et Al. followed a linear learning rate schedule , midflight changes not relevant to 125M model and  our small dataset
      optimizer=optimizer,
      num_warmup_steps=int(total_training_steps*0.05), #TODO set as a fraction of training steps=> going with 5%
      num_training_steps=total_training_steps
   )
  return optimizer, lr_scheduler

# Evaluation

https://huggingface.co/learn/nlp-course/chapter3/4?fw=pt#the-evaluation-loop
"Since we want to evaluate the model regularly on the validation set during training, let’s write a function for that as well. It just runs through the evaluation dataloader and gathers all the losses across processes:"

In [ ]:
from tqdm.notebook import tqdm
def evaluate(model,eval_dataloader,device):
  '''evaluate model to get loss and perplexity'''
  model.eval()
  losses = []
  for step, batch in tqdm(enumerate(eval_dataloader,start=1),total=len(eval_dataloader)):
     batch_on_device = {k: v.to(device) for k, v in batch.items()}
     with torch.no_grad():
          outputs = model(batch_on_device["input_ids"], labels=batch_on_device["input_ids"])
     losses.append(outputs.loss)#losses.append(accelerator.gather(outputs.loss))### change
     '''if step %100==0:
            print({"step":step,"loss":(outputs.loss).item(),"loss size":(outputs.loss).size()})'''
  loss = torch.mean(torch.stack(losses))#torch.mean(torch.cat(losses)) cant concat 0 dim tensors
  try:
      perplexity = torch.exp(loss)
  except OverflowError:
      perplexity = float("inf")
  return loss, perplexity.item()

##Early Stopping
class adapted with minimal changes from : https://gist.github.com/stefanonardo/693d96ceb2f531fa05db530f3e21517d

In [ ]:
import numpy as np

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        '''params:str mode: Indicates whether to minimize ('min') or maximize ('max') the monitored metric.
                 float min_delta: The minimum change required to qualify as an improvement in the monitored metric.
                 int patience: The number of epochs to wait for an improvement before stopping the training process.
                 bool percentage: If True, min_delta is interpreted as a percentage change rather than an absolute value.'''
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        # If patience is set to 0, always consider the current metric as the best
        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        # If the metric is NaN, consider it as a signal to stop early
        if torch.isnan(metrics):#np.isnan(metrics):# consider using torch.isnan(metrics) :
            return True

        # Check if the current metric is better than the best
        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        # If the number of bad epochs exceeds patience, stop early
        if self.num_bad_epochs >= self.patience:
            print('Terminating because of early stopping!')
            return True
        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('Mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                # Check if a is smaller than (best - min_delta)
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                # Check if a is larger than (best + min_delta)
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                # Check if a is smaller than (best - (best * min_delta / 100))
                self.is_better = lambda a, best: a < best - (best * min_delta / 100)
            if mode == 'max':
                # Check if a is larger than (best + (best * min_delta / 100))
                self.is_better = lambda a, best: a > best + (best * min_delta / 100)


In [ ]:
#Example
es = EarlyStopping(patience=5)


num_epochs = 100
for epoch in range(num_epochs):
      train_one_epoch(model, data_loader)  # train the model for one epoch.
      metric = eval(model, data_loader_dev)  # evalution on dev set.
      if es.step(metric):
          break  # early stop criterion is met, we can stop now


NameError: name 'train_one_epoch' is not defined

# Logging
##TensorBoard

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter(log_dir)

####Log scalars
 # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(training_loader) + i)
####Visualize network
# Again, grab a single mini-batch of images
dataiter = iter(training_loader)
images, labels = dataiter.next()

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(net, images)
writer.flush()

IndentationError: unexpected indent (1643835284.py, line 6)

## CSV writer

In [ ]:
import csv

def log_prog(dicts, fname, curr_step, log_steps):
    '''Args:
       - list[dict] dicts: Training loop logging entries for a list of batches.
       - str fname: Path of the file to append entries to.
       - int curr_step: The training step at which the logging is happening.
       - int log_steps: This function is called every log_steps steps. curr_step%log_steps ==0
       Returns: None'''

    try: # avoid inerupting training loop
        fields = dicts[0].keys()
        if curr_step/log_steps == 1:  # Check if it's the 1st logging instance
            with open(fname, 'w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=fields)
                writer.writeheader()
                writer.writerows(dicts)
        else:
            with open(fname, 'a', newline='') as file:  # Open the file in append mode
                writer = csv.writer(file)
                for dictionary in dicts:
                  writer.writerow(dictionary.values())
    except Exception as e:
        print(f"Logging failed: {e}")

    return


#Training loop

Refactoring giant training loop code into functions (In progress)

See code used under experiments for training loops tested

In [ ]:

def train_one_step(model,device,batch,optimizer,lr_scheduler):
  '''Args: model to train one step (forward and backward pass)
                 associated optimizer and scheduler
            pt tensor batch to be passed through model
     return: current loss and learning rate for logging '''
  #Forward Pass
  batch_on_device = {k: v.to(device) for k, v in batch.items()}
  fpass_output=model(input_ids=batch_on_device["input_ids"],labels=batch_on_device["labels"]) # Forward pass: **batch ??
  logits = fpass_output.logits  #Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax):logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size))
  loss =fpass_output.loss #cross-entropy loss: loss (torch.FloatTensor of shape (1,), optional, returned when labels is provided) — Language modeling loss (for next-token prediction).
  #update logging vars
  curr_loss=loss.item()
  curr_lr=lr_scheduler.get_lr()
  #backward pass
  loss.backward(loss)
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip the gradients to prevent exploding gradients => used 1.0 to 0.3 clipping in OPT paper
  optimizer.step()
  lr_scheduler.step()
  optimizer.zero_grad()  # Clear gradients from the previous optimization step
  return curr_loss , curr_lr

In [ ]:
from tqdm.notebook import tqdm
def train_one_epoch(model,lr_scheduler,optimizer,device,epoch,train_dataloader,eval_dataloader,total_training_steps,completed_steps,log_steps,t_log,eval_steps,e_log,early_stopper,spath):
  '''Args:  model and associated lr_scheduler and optimizer being trained on device
            int epoch being trained
            tensor list train and eval dataloaders
            int steps
            fnames for log_files training and eval
            EarlyStopping early_stopper
      return  step, boolean converged '''
  progress=[]
  for step, batch in tqdm(enumerate(train_dataloader, start=1), total=total_training_steps): #iterate with progress bar where  'step'= current batch index, 'batch' contains the input data for the current batch , represent batch 0 as batch 1
    loss,lr=train_one_step(model,device,batch,optimizer,lr_scheduler)
    completed_steps += 1
    prog={"steps": completed_steps, "loss/train": loss,"lr": lr,"tensor_memory":torch.cuda.memory_allocated(device)}
    progress.append(prog)
    if (epoch*step) % (epoch*log_steps) == 0: #LOGGING progress metrics computed on train set every log_steps
      log_prog(progress,t_log,step, log_steps)
    if ((epoch*step) % (epoch*eval_steps)) == 0:# Perform model evaluation on validation data after every eval_steps, set eval_steps=len(train_dataloader) for per epoch logging (generalizes to currOPT)
            eval_loss, perplexity = evaluate(model,eval_dataloader,device)  # Evaluate the model's performance on the evaluation dataset
            result=[{"loss/eval": eval_loss, "perplexity": perplexity}]
            log_prog(result,e_log,step,eval_steps)
            progress=[]
            #####check pointing code######
            torch.save({
            'epoch': epoch,
            'completed_steps':completed_steps,
            'model_state_dict': model.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'lr':lr,
            'optimizer_state_dict': optimizer.state_dict(),
            'eval_loss': eval_loss,
            }, spath+str(completed_steps)+'.pt') #save to path +name+'step:'+completed_steps+'.pt'
            if early_stopper.step(eval_loss):
              return step ,True #converged
            model.train()  # Set the model back to training mode
  return step, False #will continue training unless last epoch

In [ ]:
def training_loop(model,lr_scheduler,optimizer,device,num_epochs,train_dataloader,eval_dataloader,total_training_steps,completed_steps,log_steps,t_log,eval_steps,e_log,early_stopper,output_dir):
  tval=0
  for epoch in range(1,num_epochs+1):
    tval,converged=train_one_epoch(model,lr_scheduler,optimizer,device,epoch,train_dataloader,eval_dataloader,total_training_steps,completed_steps,log_steps,t_log,eval_steps,e_log,early_stopper,spath=output_dir)
    if converged:
       model.save_pretrained(output_dir)
       return tval
    #else prepare for next epoch
    completed_steps=tval
    random.shuffle(train_dataloader)
    random.shuffle(eval_dataloader)
  model.save_pretrained(output_dir)
  return tval

In [ ]:
import random
#Initial params
random.seed(32) #set seed
num_train_epochs=2
context_length=128
batch_size=32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
sync_dir='' #'/home/sharedDATA/models/'
#directories
tok_dir="" #store tokenizer



cuda


In [ ]:
import pynvml
from pynvml import *  # provides access to NVIDIA's NVML (NVIDIA Management Library).enables  querying and monitoring of NVIDIA GPU devices.


def print_gpu_utilization():
    nvmlInit()  # Initializing the NVML library
    handle = nvmlDeviceGetHandleByIndex(0)  # Obtaining a handle for the GPU device at index 0
    info = nvmlDeviceGetMemoryInfo(handle)  # Retrieving memory information for the GPU device
    print(f"GPU memory occupied: {info.used//1024**2} MB.")  # Printing the GPU memory occupied in megabytes

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'pynvml'

In [ ]:
print_gpu_utilization()

GPU memory occupied: 440 MB.


In [ ]:
#load datasets
ds_dict=create_ds_dict("")
train=ds_dict["train"]["sentence"]
print("train[1]:",train[1])
random.shuffle(train)
print("train[1] post shuffle:", train[1])
val=ds_dict["validation"]["sentence"]
print("val[1]:", val[1])
random.shuffle(val)
print("val[1] post shuffle:", val[1])
test=ds_dict["test"]["sentence"]
print("test[1] post shuffle:", test[1])
random.shuffle(test)
print("test[1] post shuffle:", test[1])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1c2c54ff78a78a92/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

train[1]: kee .
train[1] post shuffle: And then Cannata and Doss order a hit... 'cause she snuffed out their old friend.
val[1]: i yeah it's more yeah .
val[1] post shuffle: Do you think he's crying, weeping violently in the depths of the sea?
test[1] post shuffle: kangaroo .
test[1] post shuffle: Yes, but I think they're not unrelated.


In [ ]:
tokenizer=create_BPE_tok(train,tok_dir)
trainDL=create_tokenized_batches(train,tokenizer,context_length,batch_size,"trainDL.csv")
batches_per_epoch=len(trainDL[:100])
total_steps=num_train_epochs*batches_per_epoch #update gradient every epoch
model=init_OPT_CLM() #OPT125M with causal LM head for training
model.to(device)


  0%|          | 0/918982 [00:00<?, ?it/s]

batches created: 28937
OPTConfig {
  "_remove_final_layer_norm": false,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "transformers_version": "4.30.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

OPTConfig {
  "_remove_final_layer_norm": false,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
 

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [ ]:
es=EarlyStopping(patience=3)
optimizer,lr_scheduler=create_optimizer_and_lr_scheduler(total_steps)
print(optimizer)
print(lr_scheduler)
valDL=create_tokenized_batches(val,tokenizer,context_length,batch_size,"valDL.csv")




AdamW (
Parameter Group 0
    betas: (0.9, 0.95)
    correct_bias: True
    eps: 1e-06
    initial_lr: 5e-05
    lr: 0.0
    weight_decay: 0.1
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/883051 [00:00<?, ?it/s]

batches created: 27806


In [ ]:
tval=training_loop(model,lr_scheduler,optimizer,device,num_epochs=5,train_dataloader=trainDL[:100],eval_dataloader=valDL,total_training_steps=total_steps,completed_steps=0,log_steps=1000,t_log="testing_train_log.csv",eval_steps=batches_per_epoch,e_log="testing_eval_log.csv",early_stopper=es,output_dir=(sync_dir+"OPT125MV1"))

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/27806 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/27806 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in training_loop:4                                                                               │
│ in train_one_epoch:19                                                                            │
│ in evaluate:7                                                                                    │
│ in <dictcomp>:7                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
tval=0
print(tval)
testDL=create_tokenized_batches(test,tokenizer,context_length,batch_size,"testDL.csv")
tloss,tppl=evaluate(model,testDL,device)
resultDict={"T-value":tval,"training loss":tloss,"training perplexity":tppl}
print(resultDict)
with open("results.csv", 'w', newline='') as file: #may throw error , while coppying and queing run of cell forgot to change fields to resultDict.keys() and writerows(resultDict) to writerow(resultDict.values())
  writer = csv.DictWriter(file, fieldnames=resultDict.keys())
  writer.writeheader()
  writer.writerow(resultDict.values())

0


  0%|          | 0/909795 [00:00<?, ?it/s]

batches created: 28668


  0%|          | 0/28668 [00:00<?, ?it/s]

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.empty_cache()